<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)  
  
1. Признаки
    1. RowNumber — индекс строки в данных
    2. CustomerId — уникальный идентификатор клиента
    3. Surname — фамилия
    4. CreditScore — кредитный рейтинг
    5. Geography — страна проживания
    6. Gender — пол
    7. Age — возраст
    8.Tenure — сколько лет человек является клиентом банка
    9.Balance — баланс на счёте
    10.NumOfProducts — количество продуктов банка, используемых клиентом
    11.HasCrCard — наличие кредитной карты
    12.IsActiveMember — активность клиента
    13.EstimatedSalary — предполагаемая зарплата
2. Целевой признак
    1. Exited — факт ухода клиента

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings("ignore")

## Подготовка данных

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
data.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [5]:
data.duplicated().sum()

0

Взгялнув на наш dataframe можем сделать вывод о том, что для дальнейший работы нам не потребуются столбцы CusctomerId, Surname и RowNumber(потому что это столбцы идентификаторы, которые не нужны для алгоритма и не имеют влияния на результат), а так же есть немного пропусков в столбце Tenure. Так как пропусков примерно 10%, это может привести к изменению результата, поэтому для точности просто удалим их.

In [6]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [7]:
data = data.dropna(subset = ['Tenure'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      9091 non-null   int64  
 1   Geography        9091 non-null   object 
 2   Gender           9091 non-null   object 
 3   Age              9091 non-null   int64  
 4   Tenure           9091 non-null   float64
 5   Balance          9091 non-null   float64
 6   NumOfProducts    9091 non-null   int64  
 7   HasCrCard        9091 non-null   int64  
 8   IsActiveMember   9091 non-null   int64  
 9   EstimatedSalary  9091 non-null   float64
 10  Exited           9091 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 852.3+ KB


Изменим тип данных в столбцах Tenure и NumOfProducts на object

In [9]:
data['Tenure'] = data['Tenure'].astype('object')
data['NumOfProducts'] = data['NumOfProducts'].astype('object')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      9091 non-null   int64  
 1   Geography        9091 non-null   object 
 2   Gender           9091 non-null   object 
 3   Age              9091 non-null   int64  
 4   Tenure           9091 non-null   object 
 5   Balance          9091 non-null   float64
 6   NumOfProducts    9091 non-null   object 
 7   HasCrCard        9091 non-null   int64  
 8   IsActiveMember   9091 non-null   int64  
 9   EstimatedSalary  9091 non-null   float64
 10  Exited           9091 non-null   int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 852.3+ KB


Разделим данные на features и target.

In [11]:
features = data.drop('Exited', axis=1)
target = data['Exited']

Разделим наши данные на обучающую, валидационную и тестовую выборки

In [12]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test,
                                                                            test_size=0.5, random_state=12345)

In [13]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5454 entries, 9344 to 5056
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      5454 non-null   int64  
 1   Geography        5454 non-null   object 
 2   Gender           5454 non-null   object 
 3   Age              5454 non-null   int64  
 4   Tenure           5454 non-null   object 
 5   Balance          5454 non-null   float64
 6   NumOfProducts    5454 non-null   object 
 7   HasCrCard        5454 non-null   int64  
 8   IsActiveMember   5454 non-null   int64  
 9   EstimatedSalary  5454 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 468.7+ KB


In [14]:
features_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1819 entries, 5170 to 7755
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      1819 non-null   int64  
 1   Geography        1819 non-null   object 
 2   Gender           1819 non-null   object 
 3   Age              1819 non-null   int64  
 4   Tenure           1819 non-null   object 
 5   Balance          1819 non-null   float64
 6   NumOfProducts    1819 non-null   object 
 7   HasCrCard        1819 non-null   int64  
 8   IsActiveMember   1819 non-null   int64  
 9   EstimatedSalary  1819 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 156.3+ KB


In [15]:
features_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1818 entries, 7445 to 2460
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      1818 non-null   int64  
 1   Geography        1818 non-null   object 
 2   Gender           1818 non-null   object 
 3   Age              1818 non-null   int64  
 4   Tenure           1818 non-null   object 
 5   Balance          1818 non-null   float64
 6   NumOfProducts    1818 non-null   object 
 7   HasCrCard        1818 non-null   int64  
 8   IsActiveMember   1818 non-null   int64  
 9   EstimatedSalary  1818 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 156.2+ KB


In [16]:
target_train.count()

5454

In [17]:
target_test.count()

1819

In [18]:
target_valid.count()

1818

In [19]:
features_train.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
9344,727,France,Female,28,2.0,110997.76,1,1,0,101433.76
3796,537,France,Male,26,7.0,106397.75,1,0,0,103563.23
7462,610,France,Male,40,9.0,0.00,1,1,1,149602.54
1508,576,France,Male,36,6.0,0.00,2,1,1,48314.00
4478,549,France,Male,31,4.0,0.00,2,0,1,25684.85


При первичном анализе данных мы избавились от ненужных столбцов, удалили строки с пропущенными значениями, проверили на явные дубликаты и разделили выборку на обучающую, валидационную и тестовую.

Применим OneHotEncoder для категориальных столбцов

In [20]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5454 entries, 9344 to 5056
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      5454 non-null   int64  
 1   Geography        5454 non-null   object 
 2   Gender           5454 non-null   object 
 3   Age              5454 non-null   int64  
 4   Tenure           5454 non-null   object 
 5   Balance          5454 non-null   float64
 6   NumOfProducts    5454 non-null   object 
 7   HasCrCard        5454 non-null   int64  
 8   IsActiveMember   5454 non-null   int64  
 9   EstimatedSalary  5454 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 468.7+ KB


In [21]:
encoder_ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
categorical = ['Tenure', 'NumOfProducts', 'Geography', 'Gender']
encoder_ohe.fit(features_train[categorical])
features_train[encoder_ohe.get_feature_names()] = encoder_ohe.transform(features_train[categorical])
features_train = features_train.drop(categorical, axis=1)

In [22]:
features_valid[encoder_ohe.get_feature_names()] = encoder_ohe.transform(features_valid[categorical])
features_valid = features_valid.drop(categorical, axis=1)
features_test[encoder_ohe.get_feature_names()] = encoder_ohe.transform(features_test[categorical])
features_test = features_test.drop(categorical, axis=1)

## Исследование задачи

In [23]:
target.sum()/target.count()

0.2039379606203938

In [24]:
1 - target.sum()/target.count()

0.7960620393796062

Данные несбаналансированны. Количество клиентов, которые отказались от услуг банка примерно в 4 раза меньше, чем тех, кто осталася. Исправим данный дисбаланс дальше, в соответствующем пункте.

Стандартизируем данные.

In [25]:
pd.options.mode.chained_assignment = None
scaler = StandardScaler()
numerical = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']
scaler.fit(features_train[numerical])
features_train[numerical] = scaler.transform(features_train[numerical])

In [26]:
features_valid[numerical] = scaler.transform(features_valid[numerical])
features_test[numerical] = scaler.transform(features_test[numerical])

Обучим дерево решений

In [27]:
best_model = None
best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = f1_score(target_valid, predictions_valid)
    if accuracy > best_accuracy:
        best_model = model
        best_accuracy = accuracy
        best_depth = depth
print("f1_score лучшей модели на валидационной выборке:", best_accuracy, "Максимальная глубина:", best_depth)  

f1_score лучшей модели на валидационной выборке: 0.5641711229946524 Максимальная глубина: 2


Обучим случайный лес

In [28]:
best_model = None
best_accuracy = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = f1_score(target_valid, predictions_valid)
        if accuracy > best_accuracy:
            best_model = model
            best_accuracy = accuracy
            best_est = est
            best_depth = depth
print("f1_score лучшей модели на валидационной выборке:", best_accuracy, "Количество деревьев:",
      best_est, "Максимальная глубина:", best_depth)    

f1_score лучшей модели на валидационной выборке: 0.5834738617200674 Количество деревьев: 20 Максимальная глубина: 10


Обучим логистическую регрессию

In [29]:
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_lr.fit(features_train, target_train)
predicted_valid = model_lr.predict(features_valid)
print('f1_score логистической регрессии:', f1_score(target_valid, predicted_valid))

f1_score логистической регрессии: 0.5137931034482758


Лучшие модель из исследования - это случайный лес с максимальной глубиной 10 и количеством деревьем 20. Однако мы провели исследование не избавившись от дисбаналанса. Посмотрим на результаты после его устранения.

## Борьба с дисбалансом

Начнем с Upsampling'а.

In [30]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    return shuffle(features_upsampled, target_upsampled, random_state=12345)

In [31]:
features_upsampled_train, target_upsampled_train = upsample(features_train, target_train, 4)

Обучим случайный лес на новых данных и найдем лучшую модель.

In [32]:
best_model = None
best_accuracy = 0
best_est = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range(1, 21):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_upsampled_train, target_upsampled_train)
        predictions_valid = model.predict(features_valid)
        accuracy = f1_score(target_valid, predictions_valid)
        if accuracy > best_accuracy:
            best_model = model
            best_accuracy = accuracy
            best_est = est
            best_depth = depth
print("f1_score лучшей модели на валидационной выборке:", best_accuracy, "Количество деревьев:",
      best_est, "Максимальная глубина:", best_depth)    

f1_score лучшей модели на валидационной выборке: 0.635897435897436 Количество деревьев: 90 Максимальная глубина: 14


Теперь разберемся с downsampling'ом.

In [33]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    return shuffle(features_downsampled, target_downsampled, random_state=12345)

In [34]:
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, 0.25)

In [35]:
target_downsampled_train

6296    0
9131    0
4643    1
3084    0
9624    1
       ..
5134    1
2018    0
2649    0
6519    1
8702    0
Name: Exited, Length: 2208, dtype: int64

Теперь то же самое для этих данных.

In [36]:
best_model_ds = None
best_accuracy_ds = 0
best_est_ds = 0
best_depth_ds = 0
for est in range(10, 101, 10):
    for depth in range(1, 21):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_downsampled_train, target_downsampled_train)
        predictions_valid = model.predict(features_valid)
        accuracy = f1_score(target_valid, predictions_valid)
        if accuracy > best_accuracy_ds:
            best_model_ds = model
            best_accuracy_ds = accuracy
            best_est_ds = est
            best_depth_ds = depth
print("f1_score лучшей модели на валидационной выборке:", best_accuracy_ds, "Количество деревьев:",
      best_est_ds, "Максимальная глубина:", best_depth_ds)

f1_score лучшей модели на валидационной выборке: 0.5920155793573515 Количество деревьев: 80 Максимальная глубина: 9


Проведя downsampling и upsampling - мы улучшили качество модели и получили f1_score больший 0.59 для увелечиния и для уменьшения выборок. Для тестирования будем импользовать случайный лес с upsampling'ом с количеством деревьев 90 и максимальной глубиной 14.

## Тестирование модели

In [37]:
best_model

RandomForestClassifier(max_depth=14, n_estimators=90, random_state=12345)

In [38]:
features_test1 = pd.concat([features_valid, features_test])
target_test1 = pd.concat([target_valid, target_test])
predicted_test = best_model.predict(features_test1)
f1_score(target_test1, predicted_test)

0.6175115207373272

In [39]:
probabilities_test = best_model.predict_proba(features_test1)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test1, probabilities_one_test)
auc_roc

0.860214510480925

AUC-ROC нашей модели существенно выше случайной, соответственно это показывают высокую точность определения положительных результатов.

Проведем проверку на адекватность с помощью DummyClassifier.

In [40]:
model_dummy = DummyClassifier()
model_dummy.fit(features_train, target_train)
predictions_dummy = model_dummy.predict(features_test1)
print("Accuracy данной модели получился", accuracy_score(target_test1, predictions_dummy))

Accuracy данной модели получился 0.7998350288699477


In [41]:
print("Accuracy нашей модели получился", accuracy_score(target_test1, predicted_test))

Accuracy нашей модели получился 0.8402529557327467


Соответственно проверка на адекватность пройдена.

Вывод.  
  
Мы исследовали модели 3 модели для несбалансированных классов. Затем избавились от дисбаланса двумя способами downsampling'ом и upsampling'ом. Обучили случайный лес для них и получили значение метрики f1_score выше того, что нам требовалось. Для финального теста была выбрана модель обученная на данных umpsampling'а(случайный лес с количеством деревьев 90 и максимальной глубиной 14). Для тестовых данных был получен результат f1_score = 0.6175115207373272. Так же была исследована метрика AUC-ROC, которая оказалась равна 0.860214510480925, что говорит нам о высокой точности определения нашей моделью положительных результатов.